In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
import seaborn as sns
import re

In [ ]:
train_df = pd.read_csv('/content/gdrive/MyDrive/CS271/data/train.csv')
train_df.head()
train_df = train_df.drop(columns=['title', 'author'])

test_df =pd.read_csv('/content/gdrive/MyDrive/CS271/data/test.csv')
test_df.head()

submit_df =pd.read_csv('/content/gdrive/MyDrive/CS271/data/submit.csv')
submit_df.head()

test_text = test_df['text'].tolist()
test_label = submit_df['label'].tolist()

test_df = pd.DataFrame([[test, label] for test, label in zip(test_text, test_label)], columns=['text', 'label'])

train_df.fillna('' , inplace=True)
test_df.fillna('' , inplace=True)

df_news = train_df.append(test_df)

## Text Cleaning

In [ ]:
# Dataset Preprocessing
import string


def text_cleaning(text):
    text = re.sub("[^a-zA-Z]", " ", text) # removing punctuation
    # text = text.translate(str.maketrans('', '', string.punctuation))
    return text.lower()

df_news['text'] = df_news['text'].apply(text_cleaning)
df_news.columns = ['id', 'text', 'labels']
print(df_news.head)
print(df_news.columns)

<bound method NDFrame.head of        id                                               text  labels
0     0.0  house dem aide  we didn t even see comey s let...       1
1     1.0  ever get the feeling your life circles the rou...       0
2     2.0  why the truth might get you fired october     ...       1
3     3.0  videos    civilians killed in single us airstr...       1
4     4.0  print  an iranian woman has been sentenced to ...       1
...   ...                                                ...     ...
5195  NaN  of all the dysfunctions that plague the world ...       0
5196  NaN  washington      gov  john kasich of ohio on tu...       1
5197  NaN  good morning   want to get california today by...       0
5198  NaN    previous   next       us marines to be deplo...       1
5199  NaN  perhaps you ve seen the new tv series whose pi...       0

[26000 rows x 3 columns]>
Index(['id', 'text', 'labels'], dtype='object')


## Train Test Split

In [ ]:
train_df, test_df = train_test_split(df_news, test_size = 0.25)
train_df, eval_df = train_test_split(train_df, test_size = 0.05)

## install simpletransformers

In [ ]:
! pip install simpletransformers
# ! pip install simpletransformers==0.32.3
# ! pip install transformers==3.1.0

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
rm apex
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [ ]:
!sh setup.sh

rm: cannot remove 'apex': Is a directory
fatal: destination path 'apex' already exists and is not an empty directory.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-0ab0tnn7
Created temporary directory: /tmp/pip-req-tracker-mv_ldanw
Created requirements tracker '/tmp/pip-req-tracker-mv_ldanw'
Created temporary directory: /tmp/pip-install-kbsu64tc
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-wtk_su2r
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-mv_ldanw'
    Running setup.py (path:/tmp/pip-req-build-wtk_su2r/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.7.0+cu101


    running egg_info
    creating /tmp/pip-req-

In [ ]:
import transformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = ClassificationArgs()
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01
model_args.early_stopping_metric = "mcc"
model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 5
model_args.evaluate_during_training_steps = 1000
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.num_train_epochs = 6

# Create a TransformerModel
model = ClassificationModel(
    'bert', 
    'bert-base-cased', 
    num_labels=2, 
    args=model_args,
    use_cuda=True
)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
model.train_model(train_df)

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7effeb18dc88>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f00300c1ba8>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    sel

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(13896, 0.2580112275082994)

# evaluate validation set

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)
print(model_outputs)


{'mcc': 0.7504798875126645, 'tp': 426, 'tn': 427, 'fp': 71, 'fn': 51, 'eval_loss': 0.8178014066812227}
[[-4.0859375   3.69140625]
 [-4.25390625  3.7890625 ]
 [-2.50976562  2.33789062]
 ...
 [ 3.88476562 -3.53320312]
 [-4.296875    3.80664062]
 [-0.59033203  0.32885742]]


# evaluate testing set


In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)
print(result)
print(model_outputs)


{'mcc': 0.7422155153252886, 'tp': 2880, 'tn': 2782, 'fp': 446, 'fn': 392, 'eval_loss': 0.8469360040664607}
[[-4.28125     3.79882812]
 [ 3.54492188 -3.29101562]
 [-4.27734375  3.80078125]
 ...
 [-3.30078125  3.09179688]
 [-3.39257812  3.17773438]
 [ 4.140625   -3.65234375]]


In [ ]:
! ls -alh 
# ! rm -rf sample_data/

total 40K
drwxr-xr-x  1 root root 4.0K Dec  2 07:31 .
drwxr-xr-x  1 root root 4.0K Dec  2 05:07 ..
drwxr-xr-x  8 root root 4.0K Dec  2 05:13 apex
drwxr-xr-x  2 root root 4.0K Dec  2 07:08 cache_dir
drwxr-xr-x  1 root root 4.0K Nov 20 17:15 .config
drwx------  5 root root 4.0K Dec  2 05:13 gdrive
drwxr-xr-x 20 root root 4.0K Dec  2 07:06 outputs
drwxr-xr-x  4 root root 4.0K Dec  2 06:27 runs
-rw-r--r--  1 root root  181 Dec  2 06:19 setup.sh
